# Vision Integration / Calibration

This short notebook should be of help to identify transformation for the robotic tool-chain and vision system in use.

In [1]:
# math and robotics
import numpy as np
import spatialmath as sm
from tqdm.notebook import tqdm, trange

# plotting 
import seaborn as sns
import matplotlib.pylab as plt

# ROS
import rospy

# hrr-cobot packages
import hrr_cobot_robot
import hrr_common
import hrr_controllers

# set printing and plotting options    
np.set_printoptions(precision=5, suppress=True)
sns.set_theme('notebook')
%matplotlib notebook

In [2]:
hrr_common.utils.set_ros_environment(ros_master = 'hrrcobotLinux54')

current hostname:	hrrcobotLinux54
current IP:      	129.187.147.74
ROS-MASTER-URI:  	http://hrrcobotLinux54:11311


In [ ]:
rospy.init_node('kinematic_checks')

Unable to register with master node [http://hrrcobotLinux54:11311]: master may not be running yet. Will keep trying.


## Prepare handles

initialize cobot and assign current tool.
Then check the output

```python
cobot.change_tool("shaftgrinder", robot_urdf_prefix="hrr_cobot.")
print(cobot)
```

to have a non-zero transformation from EE->tool frame

```
Hrr-Cobot state:
FT-data:
current force:	[-0.45813 -0.41504 -0.30762] [N]
current torque:	[0.14484 0.16644 0.06641] [Nm]
current wrench:	[-0.46836 -0.42284 -0.31476  0.1758   0.1569   0.06113] [N,Nm]
=>in contact:	False
---
Robot-data:
q:		[ -0.03685   0.01848 -89.98192   0.2006   90.08286   0.07877][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.43594 0.      0.705  ][m]
quat:		-0.00073 <[-0.00101  1.      -0.00176]>
tool-pos:	[0.43594 0.      0.705  ][m]
tool-rpy:	[179.88437  -0.08363 179.79813][°]
robot-status:	ready
---
tool set to shaft_grinder
E_p_EC:=[-0.07601 -0.07601  0.1525 ]
R_C_E:=[[ 0.70711 -0.70711  0.     ]
 [ 0.       0.       1.     ]
 [-0.70711 -0.70711  0.     ]]
---
```

In [4]:
cobot = hrr_cobot_robot.HrrCobotControl.from_ros(cobot_prefix="/hrr_cobot")

pybullet build time: May  8 2021 05:48:13


Mujoco not found on this client. Do not use this file!
Mujoco not found on this client. Do not use this file!
current implementation does not cope with trees with edge degree over 1. Select first element from: ee_linkconnect_flange_attach_joint, ee_to_tcp  
IK q1 not in cache. Regenerate  


In [10]:
cobot.change_tool("screwdriver", robot_urdf_prefix="hrr_cobot.")
print(cobot)

Hrr-Cobot state:
FT-data:
current force:	[-26.03705  -0.60596  18.91846] [N]
current torque:	[ -5.21411 -25.28082   0.84668] [Nm]
current wrench:	[  0.33878   0.32129  43.05352 -11.557   -46.68017  -0.73525] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[ 10.25831  15.5371  -91.25151   0.35197  73.15917 -33.10429][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[ 0.52451 -0.0954   0.57787][m]
quat:		0.00102 <[ 0.36862  0.92958 -0.00278]>
tool-pos:	[ 0.52451 -0.0954   0.57787][m]
tool-rpy:	[-136.73923   -0.00828  179.66092][°]
robot-status:	moving
---
tool set to screwdriver
E_p_EC:=[0.15839 0.15839 0.1475 ]
R_E_C:=[[ 0.      -0.70711  0.70711]
 [ 0.       0.70711  0.70711]
 [-1.      -0.       0.     ]]
---


In [10]:
cobot.change_tool("shaftgrinder", robot_urdf_prefix="hrr_cobot.")
print(cobot)

Hrr-Cobot state:
FT-data:
current force:	[-0.  0.  0.] [N]
current torque:	[-0.  0.  0.] [Nm]
current wrench:	[0. 0. 0. 0. 0. 0.] [N,Nm]
=>in contact:	False
---
Robot-data:
q:		[  21.54497  -10.74382 -109.03665  -76.52963    6.74127   48.00279][°]
q_dot:		[ 0.       0.       0.       0.      -0.57962  0.00001][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[ 0.42051 -0.15597  0.70865][m]
quat:		0.63844 <[0.28867 0.7086  0.08334]>
tool-pos:	[ 0.42046 -0.15619  0.7087 ][m]
tool-rpy:	[-93.05929  72.23092 123.91824][°]
robot-status:	ready
---
tool set to shaft_grinder
E_p_EC:=[0.     0.1075 0.1525]
R_C_E:=[[-1.  0. -0.]
 [-0. -0.  1.]
 [ 0.  1. -0.]]
---


[WARN]Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


## Calculate goal poses

In the following we refer to variables with ``B_`` as variables in the ``base-frame`` of the robot

Below, we can quickly calculate the desired end-effector pose ${}^{B}{\bf T}_{E}$ for a predefined goal-pose
via the ``calc_EE_pre_pose`` command.
In here, first the desired goal-pose of the tool-center-point, i.e. screwdriver/shaftgrinder-tip ${}^{B}{\bf T}_{C}$ is calculated via ``calc_goal_pose``.
The goal-pose is then moved in the base-frame along the surface-normal ${}^{B}{\bf n}$, i.e.

$$
{}^{B}{\bf T}_{C, \mathrm{pre}} = 
\begin{bmatrix}
 {\bf 0} & d {}^{B}{\bf n} \\
 {\bf 0} & 1 \\
\end{bmatrix}
{}^{B}{\bf T}_{C}
$$

for a distance $d$, (5e-2 below).

In [6]:
B_normal = np.r_[0., 0., 1.]
B_p_goal = np.r_[0.5, 0., 0.]
B_y_axis = np.r_[0., 1.0, 0.0]
safety_distance=5e-2

In [7]:
hrr_common.utils.calc_EE_pre_pose(B_normal, B_p_goal, y_axis=B_y_axis,
                                              C_p_CE=cobot.C_p_CE, R_C_E=cobot.R_C_E, safety_distance=safety_distance)

  -0.7071    0.7071    0         0.424     
   0         0         1         0.07601   
   0.7071    0.7071    0         0.2025    
   0         0         0         1         


In [8]:
T_B_C_des = hrr_common.utils.calc_goal_pose(normal=B_normal, p_location=B_p_goal, y_axis=B_y_axis)
T_B_C_des

  -1         0         0         0.5       
   0         1         0         0         
   0         0        -1         0         
   0         0         0         1         


In [9]:
T_B_C_des_pre =  sm.SE3(safety_distance * B_normal) @ T_B_C_des
T_B_C_des_pre

  -1         0         0         0.5       
   0         1         0         0         
   0         0        -1         0.05      
   0         0         0         1         


In [10]:
# transform goal-pose to ee-frame
T_C_E = sm.SE3(cobot.C_p_CE)
T_C_E.A[:3, :3] = cobot.R_C_E
T_B_E_des_pre = T_B_C_des_pre @ T_C_E
T_B_E_des_pre

  -0.7071    0.7071    0         0.424     
   0         0         1         0.07601   
   0.7071    0.7071    0         0.2025    
   0         0         0         1         


## Compare against actual data recordings

For actual scenarios, the goal-pose will be provided in an external frame, e.g. "world"
so one will obtain ${}^{W}{\bf T}_{C}$ from said source, while the robot-configuration ultimately needs
${}^{B}{\bf T}_{E}$.

While ${\bf f}({\bf q}) \mapsto {}^{B}{\bf T}_{E}$ is assumed to be known, two transformation need validation

* varify the transformations from tcp->EE, i.e. ${}^{C}{\bf T}_{E}$ 
* varify the transformation from world / camera to ${}^{W}{\bf T}_{B}$


### Identifying ${}^{C}{\bf T}_{E}$ 

drive through a fixed position and exploit symmetry in world / base-frame, i.e. rotate **around tool-tip**
Record joint-positions, denote 

$$
{}^{B}{\hat{\bf p}}_{BC} := {\bf f}({\bf q}) {}^{C}{\bf T}_{E}^{-1}[:3, 3]
$$

as the position of the robot forward-kinematics and the transformation from EE->tool-tip, the RMS-error needs to be minimized

$$ || {}^{B}{\hat{\bf p}}_{BC} - {}^{B}{\bf p}_{BC} ||$$

In [11]:
def B_p_hat_BC(q, C_p_CE=cobot.C_p_CE, R_C_E=cobot.R_C_E):
    T_C_E = sm.SE3(C_p_CE)
    T_C_E.A[:3, :3] = R_C_E
    return (cobot.FK(q) @ T_C_E.inv()).t

In [12]:
B_p_BC = T_B_E_des_pre.t

In [13]:
B_p_hat_BC(cobot.q) - B_p_BC

pybullet build time: Dec 10 2021 09:49:03


current implementation does not cope with trees with edge degree over 1. Select first element from: ee_linkconnect_flange_attach_joint, ee_to_tcp  


array([ 0.17376, -0.12951,  0.57846])

### Identifying ${}^{W}{\bf T}_{B}$

Again collect samples at different robot configurations.

Save joints and goal-poses of tool-tip at identifiable poses ${}^{W}{\bf T}_{C}$

Now the overall pose-error should be minimized, i.e. calculate the full-transformation chain from joints

$$
  {}^{W}{\hat{\bf T}}_{C} := {}^{W}{\bf T}_{B} {\bf f}({\bf q}) {}^{C}{\bf T}_{E}^{-1}
$$

and compare it against the actual data input ${}^{W}{\bf T}_{C}$

In [14]:
def W_T_hat_C(q, 
              W_p_WB=np.zeros(3), R_W_B=np.eye(3),
              C_p_CE=cobot.C_p_CE, R_C_E=cobot.R_C_E):
    T_W_B = sm.SE3(C_p_CE)
    T_W_B.A[:3, :3] = R_W_B
    T_C_E = sm.SE3(C_p_CE)
    T_C_E.A[:3, :3] = R_C_E
    return T_W_B @ cobot.FK(q) @ T_C_E.inv()

def pose_error(T_W_C, q, W_p_WB=np.zeros(3), R_W_B=np.eye(3),
              C_p_CE=cobot.C_p_CE, R_C_E=cobot.R_C_E):
    return hrr_common.utils.pose_error(W_T_hat_C(q, W_p_WB=W_p_WB, R_W_B=R_W_B, C_p_CE=C_p_CE, R_C_E=R_C_E),
                                       T_W_C)


In [15]:
pose_error(T_B_C_des_pre, cobot.q)

array([-0.17376,  0.02252, -0.57846,  1.57224,  0.00351, -0.78741])

## Lab recordings

### Screwdriver

q:		[ 10.25831  15.5371  -91.25151   0.35197  73.15917 -33.10429][°]

In [14]:
q1 = np.copy(cobot.q)
T1 = cobot.T_B_C_robot
np.save('Untitled Folder/screwdriver_ident_lab_01.npy', 
        dict(q=np.copy(cobot.q), T_B_C=cobot.T_B_C_robot, T_E_C=cobot.T_E_C_robot.copy()))


In [27]:
cobot.init_sns_vel()

In [51]:
# v_test = np.zeros(6)
# for t in trange(2000):
#     v_test[3] = 3e-2
#     cobot.update(u_cmd=v_test, u_cmd_frame="hrr_cobot.screwdriver_tip", sleep=True)
# cobot.stop()

In [25]:
cobot.move_to_joint_pose(q1)

In [31]:
q =np.deg2rad([-24.851, 22.239, -82.488, 0.317, 75.411, 109.694]) 
np.save('Untitled Folder/screwdriver_ident_lab_03.npy', 
        dict(q=q, T_B_C=cobot.T_B_C_robot, T_E_C=cobot.T_E_C_robot.copy()))

In [34]:
A = np.load('./ee_ident/screwdriver_ident_lab_01.npy', allow_pickle=True).item()
C = np.load('./ee_ident/screwdriver_ident_lab_03.npy', allow_pickle=True).item()

In [47]:
 (cobot.FK(A['q']) @ cobot.T_E_C_robot).t, \
(cobot.FK(B['q']) @ cobot.T_E_C_robot).t, \
(cobot.FK(q) @ cobot.T_E_C_robot).t


(array([0.51769, 0.12761, 0.42904]),
 array([0.51776, 0.12754, 0.42902]),
 array([0.52122, 0.01675, 0.43154]))

In [50]:
cobot.T_E_C_robot

   3.462e-12 -0.7071    0.7071    0.1584    
   3.462e-12  0.7071    0.7071    0.1584    
  -1         0         4.897e-12  0.1475    
   0         0         0         1         


In [49]:
dx = 0.5 * ((cobot.FK(q) @ cobot.T_E_C_robot).t - (cobot.FK(A['q']) @ cobot.T_E_C_robot).t)
dx

array([ 0.00177, -0.05543,  0.00125])

In [45]:
A['q'], B['q'], C['q']

(array([ 0.17904,  0.27117, -1.59264,  0.00614,  1.27687, -0.57778]),
 array([-0.15832,  1.17082, -0.14015,  0.00607,  1.83169,  0.58175]),
 array([-0.15832,  1.17082, -0.14015,  0.00607,  1.83169,  0.58175]))